In [1]:
"""Modelling share classes in LUSID

Attributes
----------
transactions
adjust holdings
holdings
aggregation
quotes
instrument definitions
"""

'Modelling share classes in LUSID\n\nAttributes\n----------\ntransactions\nadjust holdings\nholdings\naggregation\nquotes\ninstrument definitions\n'

## The Challenge

When you offer funds in multiple currencies or with multiple share classes it can be challenging to ensure that all funds are aligned. This leads to the potential risk that some funds may drift from the core mandate of the original base fund. In addition it leads to wasted time and resources in keeping the funds in-sync with each other.

## The Solution

Rather than maintaining a separate fund for each currency or share class and spending time ensuring that they all reconcile with each other, LUSID allows you to create a single base fund to hold the core transactions that generate the fund's returns.

Your funds offered in additional currencies can hold units in this core fund and any transactions that occur in the base fund will be propogated to your foreign currency funds.

Furthermore you can then apply appropriate hedging activity on each additional currency fund without worrying about it affecting the base fund in any way.

In this tutorial you will walk through how to set this up from a completely blank LUSID environment by:

1) Setting up a Scope to hold your portfolios

2) Creating your Instrument Universe 

3) Creating your base fund portfolio

4) Setting your initial holdings

5) Valuing your base fund

6) Securitising your base fund so that it can be held by your foreign currency funds

7) Creating your foreign currency portfolios

8) Adding units of the base fund to your currency portfolios

9) Adding appropriate hedging transactions for each foreign currency fund

10) Valuing your foreign currency funds

11) Adding additional transactions to the base fund

12) Re-valuing the base fund

13) Re-valuing the foreign currency funds to see how the transaction on the base fund is propogated 

*First things first run the cell below to import the libraries and authenticate your LUSID client*

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.4916.0


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Setting up a Scope to hold your portfolios

You will need to create a name for the scope that you will use for the fund. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [3]:
# Give the scope a unique identifier
scope_id = import_data.create_scope_id()
# Give the scope a descriptive name
scope_name = 'UK_High_Growth_Equities_Fund'
# Join the two together to get the full scope name
scope = '{}_{}'.format(scope_name, scope_id)
prettyprint.heading('Scope', scope)

Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02


![Scope](img/multiplecurrencies-scope.gif)

## 2) Creating your Instrument Universe 

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/instruments).

*Run the cell below to import your instrument universe*

In [4]:
# Import your instrument universe from a CSV file
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,nan
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.00
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.88
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,nan
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,nan
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,nan
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,nan
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,nan


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument)

As part of this definition you will attach identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Schemes Should I Use With LUSID?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid)

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [5]:
# Initialise a dictionary to hold your instrument definitions
instrument_definitions = {}

# Set the mapping between your identifier columns in the CSV and the available LUSID identifiers
identifier_columns = {
    'isin': 'Isin',
    'figi': 'Figi',
    'ticker': 'Ticker',
    'client_internal': 'ClientInternal'
}

# Iterate over your instrument universe
for index, instrument in instrument_universe.iterrows():

    # Initialise your set of identifiers for this instrument
    identifiers = {}
    
    # Populate your set of identifiers
    for identifier_column, identifier_lusid in identifier_columns.items():
        identifiers[identifier_lusid] = models.InstrumentIdValue(
            value=instrument[identifier_column])
        
    # Create the definition for your instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers
    )

# Call LUSID to upsert your instrument defintions as a batch
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
0,Arcosa_NYSE_ACA,BBG00JGMWFQ5,LUID_T44OEB6L
1,TrinityIndustries_NYSE_TRN,BBG000BVL406,LUID_XDOYC25R
2,Apple_Nasdaq_AAPL,BBG000B9XVV8,LUID_QX4NW4B1
3,USTreasury_6.875_2025,BBG000DQQNJ8,LUID_AH032RI8
4,USTreasury_2.00_2021,BBG00FN3B5K8,LUID_RVMG1IHT
5,Cigna_NYSE_CI,BBG00KXXK940,LUID_KI616A6T
6,Trex_NYSE_TREX,BBG000BTGM43,LUID_QNTBLJCI
7,Amazon_Nasdaq_AMZN,BBG000BVPXP1,LUID_8UQAQ0XE
8,ExpressScripts_NYSE_ESRX,BBG000C16621,LUID_K38WBDIV


![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Creating your base fund portfolio

To create a portfolio you need to give it a name and code.

*Run the cell below to give your base fund portfolio a name and code*

In [6]:
# Give your portfolio a name & code
base_portfolio_name = '{}_base_fund'.format(scope_name)
base_portfolio_code = str(uuid.uuid4())
prettyprint.heading('Portfolio Name', base_portfolio_name)
prettyprint.heading('Portfolio Code', base_portfolio_code)

Portfolio Name: UK_High_Growth_Equities_Fund_base_fund
Portfolio Code: 77f82cd2-3ae6-49bb-829c-021d6193eff3


Now that you have decided on the name and unique code for your portfolio you can create your base fund. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/portfolios).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 365 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [7]:
# The date your portfolio was first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add your portfolio to LUSID
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_code,
    base_currency='USD',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 77f82cd2-3ae6-49bb-829c-021d6193eff3
Portfolio Effective From: 2019-07-30 02:13:53.135729+00:00
Portfolio Created On: 2020-07-29 02:13:53.344413+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Setting your initial holdings

Now that you have your instrument universe populated and base fund portfolio you can load your initial holdings into your base fund. In this case you will import your holdings from a CSV file.

*Run the cell below to import your take on balances*

In [8]:
#Import your holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD


You can add these holdings to LUSID by setting the holdings on your base fund. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The Effect of Holding Adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio). These holdings will be effective as of 4 days ago. This gives you some room to add recent transactions and conduct valuations.

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set the holdings on your base fund*

In [9]:
# Initialise a list to hold your holding adjustments
holding_adjustments = []

# Set the effective date of these holdings to be 4 days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

# Iterate over your holdings
for index, holding in holdings.iterrows():
    
    # Create a holding adjustment for this holding
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set your holdings 
response = api_factory.build(lusid.api.TransactionPortfoliosApi).adjust_holdings(
    scope=scope,
    code=base_portfolio_code,
    effective_at=holdings_effective_date,
    adjust_holding_request=holding_adjustments)

# Pretty print the response 
prettyprint.set_holdings_response(response, scope, base_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 77f82cd2-3ae6-49bb-829c-021d6193eff3
Holdings Effective From: 2019-07-30 02:13:53.135729+00:00
Holdings Created On: 2020-07-29 02:13:53.761392+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Valuing your base fund

### a) Upserting market quotes to use in your valuation

With holdings in your base fund you can now attempt to produce a value for the fund. To value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation). 

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [10]:
# Import market data for the day that the holdings of the fund are effective from
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,1622.65,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,170.80,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.00,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,142.36,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

To do this you will create a function which allows you to easily upsert quotes from an imported CSV file.

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to define your function*

In [11]:
def upsert_quotes_from_data_frame(scope, data_frame, date, instrument_identifier, lusid_identifier, quote_type):
    """
    This function takes quotes from a data_frame and upserts them into LUSID
    
    param scope (str): The LUSID scope to upsert the quotes into
    param data_frame (Pandas DataFrame): The name of the DataFrame that the quotes are in
    param date (Datetime): The datetime to upsert the quotes at
    param instrument_identifier (str): The column in the dataframe that contains the instrument identifiers
    param lusid_identifier (str): The matching LUSID instrument identifier
    param quote_type (str): The type of quote
    """
    
    # Check that the quote type is supported
    supported_quote_types = ['price', 'fx_rate']
    if quote_type not in supported_quote_types:
        return "Quote type not supported"
    
    # Check that the identifier column exists
    if instrument_identifier not in data_frame.columns:
        return "Instrument identifier column missing"
    
    # Initialise an empty instrument quotes list to hold the quotes
    instrument_quotes = {}
    
    quote_type_values = {
        'price': {
            'quote_type': 'Price',
            'price_side': 'Mid',
            'value': 'price',
        },
        'fx_rate': {
            'quote_type': 'Rate',
            'price_side': 'Mid',
            'value': 'rate',
        }
    }
    
    # Iterate over the quotes
    for index, quote in data_frame.iterrows():

        if quote_type == 'price':
            unit = quote['currency']
        else:
            unit = 'rate'
        
        # Add the quote to the list of upsert quote requests
        instrument_quotes[quote[instrument_identifier]] = models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider='DataScope',
                    instrument_id=quote[instrument_identifier],
                    instrument_id_type=lusid_identifier,
                    quote_type=quote_type_values[quote_type]['quote_type'],
                    field=quote_type_values[quote_type]['price_side']
                ),
                effective_at=date),
            metric_value=models.MetricValue(
                value=quote[quote_type_values[quote_type]['value']],
                unit=unit),
            lineage='InternalSystem'
        )

        
    # Upsert the quotes into LUSID
    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=scope,
        request_body=instrument_quotes)

    # Pretty print the response
    #prettyprint.upsert_quotes_response(response)
    return prettyprint.upsert_quotes_response(response)

Now you can use your function to upsert the price quotes.

*Run the cell below to upsert the market data quotes into LUSID*

In [12]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=prices, 
    date=holdings_effective_date, 
    instrument_identifier='instrument_figi', 
    lusid_identifier='Figi', 
    quote_type='price')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:54.147940+00:00,None,DataScope,None,BBG000BVPXP1,Figi,Price,Mid,1622.65,USD,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:54.147940+00:00,None,DataScope,None,BBG000B9XVV8,Figi,Price,Mid,170.80,USD,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:54.147940+00:00,None,DataScope,None,BBG00FN3B5K8,Figi,Price,Mid,98.00,USD,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:54.147940+00:00,None,DataScope,None,BBG000DQQNJ8,Figi,Price,Mid,142.36,USD,Success


![Analytic-Stores](img/multiplecurrencies-quotestore.gif)

### b) Valuing your fund using quotes via aggregation

Now that the quotes have been added to the quote store you can value your base fund portfolio. You do this in LUSID via an aggregation. The logic for a valuation via aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value the base fund*

In [13]:
# Specify the inline recipe used to perform the aggregation
inline_recipe = models.ConfigurationRecipe(
    scope="User",
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               field='Mid'),
           models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               field='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Rate',
               field='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
    )
)

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Instrument/default/LusidInstrumentId',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
    ],
    group_by=[
        'Instrument/default/LusidInstrumentId'
    ])

# Call LUSID to perform the aggregation
response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
    scope=scope,
    code=base_portfolio_code,
    aggregation_request=aggregation_request)

# Pretty print the response
prettyprint.aggregation_responses_generic_df([response])

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Sum(Holding/default/Units),Sum(Holding/default/Cost),Sum(Holding/default/PV),currency
0,LUID_8UQAQ0XE,Amazon_Nasdaq_AMZN,"5,000.00","7,750,000.00","8,113,250.00",USD
1,LUID_QX4NW4B1,Apple_Nasdaq_AAPL,"49,567.00","9,417,730.00","8,466,043.60",USD
2,LUID_RVMG1IHT,USTreasury_2.00_2021,"121,543.00","12,063,142.75","11,911,214.00",USD
3,LUID_AH032RI8,USTreasury_6.875_2025,"98,444.00","13,878,635.12","14,014,487.84",USD


![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 6) Securitising your base fund so that it can be held by your foreign currency funds

### a) Securitising the base fund with 40,000 units

Now that you have created and valued your base fund, you need to securitise it so that it can be held by other portfolios. Before you create the instrument that allows other portfolios to hold the base fund you need a way to hold the total number of units of the fund available to be held.

To do this you can make use of a property. Read more about properties in the [LUSID Knowledge Base: What is a Property?](https://support.lusid.com/what-is-a-property).

For further usage of the Create Property Definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold the total circulation of your base fund*

In [14]:
# Create your request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='system', code='number'))

# Call LUSID to create uour new property
response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    create_property_definition_request=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print your key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

You can now create an securitise the base fund portfolio as an instrument, allowing it to be held by other portfolios. Here you are giving it 40,000 units.

*Run the cell below to securitise the base fund*

In [15]:
# Create the instrument definition for the base fund
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': models.InstrumentIdValue(
        value=base_portfolio_code)},
    properties=[
        models.ModelProperty(
            key=circulation_property_key,
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=40000)
            )
        )
    ],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_code)
)

# Call LUSID to upser the instrument
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body={
        'securitise-base-fund':instrument_definition
    }
)

# Pretty print the response
prettyprint.instrument_response(response, identifier='ClientInternal')

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,securitise-base-fund,77f82cd2-3ae6-49bb-829c-021d6193eff3,LUID_X5M03SOH


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculating the unit price from aggregation and storing it in the quote store

So that you can accurately value any portfolios that hold the newly created base fund instrument, you need to store the value of a unit of the base fund in the quotes store. To do this you can re-value the base fund portfolio as you did earlier. The only difference is that in this case you are grouping all the holdings together to get the total value of the fund.

*Run the cell below to get the total value of the fund*

In [16]:
# Create the aggregation request, using the recipe defined earlier
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='Value'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

# Call LUSID to aggregate and value the base fund
response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
    scope=scope,
    code=base_portfolio_code,
    aggregation_request=aggregation_request)

# Set the total value and total cost of the base fund when it was first created
base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

prettyprint.heading("Base Fund Total Value", str(base_fund_total_value))
prettyprint.heading("Base Fund Total Cost", str(base_fund_total_cost))

Base Fund Total Value: 42504995.44
Base Fund Total Cost: 43109507.87


Now that you have the total value of the fund you need to divide this by its total circulation to get the unit price of the base fund. 

*Run the cell below to calculate the unit price of the base fund and upsert this quote into the quotes store*

In [17]:
# Get the base fund instrument
base_fund_instrument = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_code,
    property_keys=[circulation_property_key])

# Get the unique Lusid Instrument Id from the base fund instrument
luid = base_fund_instrument.lusid_instrument_id
# Get the total circulation from the base fund instrument
base_fund_total_circulation = round(base_fund_instrument.properties[0].value.metric_value.value, 0)
# Calculate the unit price of the base fund from the total value and the total circulation
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        quote_series_id=models.QuoteSeriesId(
            provider='DataScope',
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            field='Mid'
        ),
        effective_at=holdings_effective_date
    ),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    lineage='InternalSystem'
)

# Call LUSID to upsert the quote
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=scope,
    request_body={"base_fund": instrument_quote})

# Pretty print the total value, circulation and unit price of the fund
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
# Pretty print the response from LUSID

prettyprint.upsert_quotes_response(response)

Base Fund Unit Price: 1062.62


,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:56.062598+00:00,None,DataScope,None,LUID_X5M03SOH,LusidInstrumentId,Price,Mid,"1,062.62",USD,Success


![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 7) Creating your foreign currency portfolios

Now that you've succesfully securitised the base fund and associated it with a unit price, you can create additional portfolios to hold the fund. In this case you want to offer the fund in five different currencies.

Note that each portfolio is created with the appropriate base currency.

*Run the cell below to create a portfolio for each currency you'd like to offer the fund in*

In [18]:
# Define the 5 fund currencies to offer the fund in
fund_currencies = {
    'AUD': None,
    'GBP': None,
    'USD': None,
    'EUR': None,
    'JPY': None
}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # Create the portfolio name & code for this currency
    portfolio_name = '{} offering of {}'.format(currency, base_portfolio_name)
    portfolio_code = str(uuid.uuid4())
    fund_currencies[currency] = portfolio_code
    
    # Build the request to create your portfolio in this currency
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_code,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create your portfolio
    portfolio_response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 52716a42-5b33-4f31-98b0-5b0914ae3b34
Portfolio Effective From: 2019-07-30 02:13:53.135729+00:00
Portfolio Created On: 2020-07-29 02:13:56.345966+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 0b13815d-4f56-4787-afb4-ceb0ca644049
Portfolio Effective From: 2019-07-30 02:13:53.135729+00:00
Portfolio Created On: 2020-07-29 02:13:56.754243+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: cdd12ae6-fbce-4ae2-8648-0ce70ff82549
Portfolio Effective From: 2019-07-30 02:13:53.135729+00:00
Portfolio Created On: 2020-07-29 02:13:57.054669+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 9448d983-c865-4ad6-9d47-f92d8135256b
Portfolio Effective From: 2019-07-30 02:13:53.135729+00:00
Portfolio Created On: 2020-07-29 02:13:57.354745+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 52

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 8) Adding units of the base fund to your currency portfolios

Now that the portfolios have been created, you can add units of the base fund to them.

To do this you need to know the exchange rate at the time the currency portfolios take on the base fund units. This allows you to calculate the portfolio cost of the holdings.

In this case you will import the foreign exchange rates from a CSV file.

*Run the cell below to import the foreign exchange rates*

In [19]:
# Import the FX rates
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.31
1,AUD/USD,0.72
2,EUR/USD,1.14
3,JPY/USD,0.01


Now that you have imported the FX rates you can upsert them into the quote store.

*Run the cell below to upsert the FX rates into the quote store*

In [20]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=fx_rates, 
    date=holdings_effective_date, 
    instrument_identifier='pair', 
    lusid_identifier='CurrencyPair', 
    quote_type='fx_rate')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.881906+00:00,None,DataScope,None,GBP/USD,CurrencyPair,Rate,Mid,1.31,rate,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.881906+00:00,None,DataScope,None,AUD/USD,CurrencyPair,Rate,Mid,0.72,rate,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.881906+00:00,None,DataScope,None,EUR/USD,CurrencyPair,Rate,Mid,1.14,rate,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.881906+00:00,None,DataScope,None,JPY/USD,CurrencyPair,Rate,Mid,0.01,rate,Success


You will also use the inverse FX rates for calculations later and thus will want to add them to the quote store too.

*Run the cell below to generate the inverse rates*

In [21]:
fx_rates_inverse = fx_rates.copy(deep=True)
fx_rates_inverse['pair'] = fx_rates_inverse['pair'].apply(lambda x: '/'.join(x.split('/')[::-1]))
fx_rates_inverse['rate'] = fx_rates_inverse['rate'].apply(lambda x: 1/x)
fx_rates_inverse.head()

,pair,rate
0,USD/GBP,0.76
1,USD/AUD,1.39
2,USD/EUR,0.88
3,USD/JPY,111.11


*Run the cell below to upsert the rates into LUSID*

In [22]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=fx_rates_inverse, 
    date=holdings_effective_date, 
    instrument_identifier='pair', 
    lusid_identifier='CurrencyPair', 
    quote_type='fx_rate')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.960844+00:00,None,DataScope,None,USD/GBP,CurrencyPair,Rate,Mid,0.76,rate,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.960844+00:00,None,DataScope,None,USD/AUD,CurrencyPair,Rate,Mid,1.39,rate,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.960844+00:00,None,DataScope,None,USD/EUR,CurrencyPair,Rate,Mid,0.88,rate,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:13:57.960844+00:00,None,DataScope,None,USD/JPY,CurrencyPair,Rate,Mid,111.11,rate,Success


Now that you have loaded the FX rates into the quote store you can add units of the securitised base fund to each of the currency portfolios. In this case you will give each currency portfolio the same amount of units.

Note that here you are pulling the FX rates back out of the quote store using the Get Quotes API call. For further usage of the Get Quotes API call refer to the [LUSID API Docs: Get Quotes](https://docs.lusid.com/#operation/GetQuotes).

For further usage of the Adjust Holdings API call refer to the [LUSID API Docs: Adjust Holdings](https://docs.lusid.com/#operation/AdjustHoldings).

*Run the cell below to adjust the holdings of the currency portfolios to hold the base fund*

In [23]:
# Work out the base fund units to give to each portfolio, dividing equally amongst them
base_fund_units_per_portfolio = base_fund_total_circulation/len(fund_currencies)

# Work out the total cost for each portfolio in the currency of the base fund (USD)
total_cost_per_portfolio = base_fund_total_cost/len(fund_currencies)

# Iteratve over each currency
for currency, currency_portfolio_code in fund_currencies.items():
    
    # If the currency is the same as the base fund, skip getting an FX rate
    if currency!='USD':
        # Get the appropriate FX rate from the quotes store
        fx_rate = api_factory.build(lusid.api.QuotesApi).get_quotes(
            scope=scope,
            effective_at=holdings_effective_date,
            request_body={
                'CurrencyPair': models.QuoteSeriesId(
                        provider='DataScope',
                        instrument_id='{}/USD'.format(currency),
                        instrument_id_type='CurrencyPair',
                        quote_type='Rate',
                        field='Mid')
            }).values['CurrencyPair'].metric_value.value
    else:
        # Use an FX rate of 1 if the currency is the same as the base fund
        fx_rate = 1
    
    # Initialise an empty list for the holding adjustments
    holding_adjustments = []
    
    # Add a holding adjustment to the list for this currency portfolio
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/ClientInternal': base_portfolio_code},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=base_fund_units_per_portfolio,
                        cost=models.CurrencyAndAmount(
                            amount=total_cost_per_portfolio,
                            currency='USD'),
                        portfolio_cost=total_cost_per_portfolio/fx_rate, 
                        price=base_fund_total_cost/base_fund_total_circulation)
                ]
        )
    )

    # Call LUSID to adjust your holdings
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).adjust_holdings(
        scope=scope,
        code=currency_portfolio_code,
        effective_at=holdings_effective_date,
        adjust_holding_request=holding_adjustments)
    
    # Pretty print the response from LUSID
    prettyprint.set_holdings_response(response, scope, currency_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 52716a42-5b33-4f31-98b0-5b0914ae3b34
Holdings Effective From: 2019-07-30 02:13:53.135729+00:00
Holdings Created On: 2020-07-29 02:13:58.355140+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 0b13815d-4f56-4787-afb4-ceb0ca644049
Holdings Effective From: 2019-07-30 02:13:53.135729+00:00
Holdings Created On: 2020-07-29 02:13:59.023040+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: cdd12ae6-fbce-4ae2-8648-0ce70ff82549
Holdings Effective From: 2019-07-30 02:13:53.135729+00:00
Holdings Created On: 2020-07-29 02:13:59.734562+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 9448d983-c865-4ad6-9d47-f92d8135256b
Holdings Effective From: 2019-07-30 02:13:53.135729+00:00
Holdings Created On: 2020-07-29 02:14:00.421805+00:00



![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 9) Adding appropriate hedging transactions for each foreign currency fund

Even though you are offering your base fund in different currencies, you don't want fluctuations in the exchange rate to affect the performance of each currency portfolio. You therefore plan to hedge against the foreign exchange rate risk. Each hedge comes in the form of a forward FX sell contract which you plan to roll over weekly. 

Read more about creating bespoke instruments in the [LUSID Knowledge Base: Creating Bespoke Instruments](https://support.lusid.com/creating-bespoke-instruments). 

*Run the cell below to create your first hedging contracts*

In [24]:
# Initialise a dictionary to hold the hedging contracts
hedge_contracts = {}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # If the currency matches the base fund there is no need for a hedge
    if currency == 'USD':
        continue
        
    # Get the current foreign exchange rate for the first hedge contract
    fx_rate = api_factory.build(lusid.api.QuotesApi).get_quotes(
        scope=scope,
        effective_at=holdings_effective_date,
        request_body={
            'CurrencyPair': models.QuoteSeriesId(
                    provider='DataScope',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    field='Mid')
        }).values['CurrencyPair'].metric_value.value

    # Create the bespoke hedge contract definition
    hedge_contract = {
        "contract_id": "tid_{}".format(uuid.uuid4()),
        "type": "FwdFxSell",
        "legs": 2,
        "transaction_date": holdings_effective_date.isoformat(),
        "settlement_date": (holdings_effective_date+timedelta(days=7)).isoformat(),
        "strike_price": 1/fx_rate,
        "leg1": {
            "type": "sell",
            "currency": "USD",
            "amount": round(total_cost_per_portfolio, 0)
        },
        "leg2": {
            "type": "buy",
            "currency": currency,
            "amount": round(total_cost_per_portfolio, 0) / fx_rate
        }
    }

     # Create the definition for your instrument, attaching the bespoke contract
    hedge_contracts['{}_hedge'.format(currency)] = models.InstrumentDefinition(
        name='hedge_{}'.format(currency),
        identifiers={
            'ClientInternal': models.InstrumentIdValue(
                value=hedge_contract["contract_id"])},
        definition=models.InstrumentEconomicDefinition(
            instrument_format='JSON',
            content=json.dumps(hedge_contract)
        )
    )
        
# Call LUSID to upsert your instrument defintions
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=hedge_contracts)

# Pretty print the response from LUSID
prettyprint.instrument_response(response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,GBP_hedge,tid_64422e3f-c52f-4cb8-953a-c38cc3394d13,LUID_HNJUECZO
1,JPY_hedge,tid_faaa5072-9537-4dbf-bf7f-e62c5e4ce1e1,LUID_T0PNV9AK
2,EUR_hedge,tid_d34b65f2-4577-4e07-abfe-26cb39932062,LUID_3SAI7Q01
3,AUD_hedge,tid_89fcbb0a-b99b-45ed-9e95-7b62c261898c,LUID_XSQRTPVG


Now that you've added your hedge contracts to your instrument master inside LUSID you can log transactions against them. Read more about transactions in the [LUSID Knowledge Base: What is a Transaction?](https://support.lusid.com/what-is-a-transaction). 

For further usage of the Upsert Transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://fbn-prd.lusid.com/docs/api#operation/UpsertTransactions).

*Run the cell below to upsert a hedging transaction for each currency portfolio*

In [25]:
# Iterate over the currencies
for currency, currency_portfolio_code in fund_currencies.items():
    
    if currency == 'USD':
        continue
        
    # Initialse a list to hold the hedging transaction
    hedge_transactions = []

    # Retrieve the bespoke hedge contract from the hedging contract definition
    contract = json.loads(api_factory.build(lusid.api.InstrumentsApi).get_instrument(
        identifier_type='ClientInternal',
        identifier=hedge_contracts['{}_hedge'.format(currency)].identifiers['ClientInternal'].value
    ).instrument_definition.content)

    # Build and add the hedge transaction for this currency to the list of transactions
    hedge_transactions.append(
        models.TransactionRequest(
            transaction_id=contract['contract_id'],
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal': contract['contract_id']},
            transaction_date='T'.join(contract['transaction_date'].split(' ')),
            settlement_date='T'.join(contract['settlement_date'].split(' ')),
            units=1,
            transaction_price=models.TransactionPrice(
                price=0,
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=0,
                currency=currency),
            transaction_currency='USD',
            source='Client',
        )
    )
    
    # Upsert the transaction into LUSID
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
        scope=scope,
        code=currency_portfolio_code,
        transaction_request=hedge_transactions
        )
  
    # Pretty print the response from LUSID
    prettyprint.transactions_response(
        response, 
        scope, 
        currency_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 52716a42-5b33-4f31-98b0-5b0914ae3b34
Transactions Effective From: 2020-07-25 02:13:53.451114+00:00
Transactions Created On: 2020-07-29 02:14:02.568076+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 0b13815d-4f56-4787-afb4-ceb0ca644049
Transactions Effective From: 2020-07-25 02:13:53.451114+00:00
Transactions Created On: 2020-07-29 02:14:02.974173+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 9448d983-c865-4ad6-9d47-f92d8135256b
Transactions Effective From: 2020-07-25 02:13:53.451114+00:00
Transactions Created On: 2020-07-29 02:14:03.426397+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 5246482d-c376-4e49-a5b8-cd5a6ddf3ecc
Transactions Effective From: 2020-07-25 02:13:53.

## 10) Valuing your foreign currency funds

With the hedging transactions added to each foreign currency portfolio you now want to value them. To do this you first need to value the currency forward contracts.

### a) Valuing the currency forward contracts

In this case you will value these contracts by taking into account the interest rate available in each currency. 

*Run the cell below to load interest rates from a CSV file*

In [26]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.03
1,AUD,0.02
2,EUR,-0.00
3,JPY,0.00
4,GBP,0.01


Define a function to value the currency forwards.

*Run the cell below to define the valuation function*

In [27]:
def value_currency_forwards(scope, fund_currencies, transaction_date, valuation_date, interest_rates):
    
    quote_requests = {}

    # Iterate over each currency
    for currency, currency_portfolio_code in fund_currencies.items():

        # Ensure that the currency is not the same as the base fund currency
        if currency!='USD':
            # Get the current FX rate for buying USD from the portfolio currency
            fx_rate = api_factory.build(lusid.api.QuotesApi).get_quotes(
                scope=scope,
                effective_at=valuation_date,
                request_body={
                    'CurrencyPair': models.QuoteSeriesId(
                            provider='DataScope',
                            instrument_id='{}/USD'.format(currency),
                            instrument_id_type='CurrencyPair',
                            quote_type='Rate',
                            field='Mid')
                }).values['CurrencyPair'].metric_value.value
        else:
            # If the currency is the same as the base fund use an fx_rate of 1
            fx_rate = 1

        # Get all transactions for the currency portfolio
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_transactions(
            scope=scope,
            code=currency_portfolio_code,
            from_transaction_date=transaction_date,
            to_transaction_date=transaction_date,
            filter="type eq 'Buy'")

        # Ensure that the at least one transaction exists
        if len(response.values) == 0:
            continue

        # Get the hedging transaction which should be the only transaction
        hedge_transaction = response.values[0]

        # Get the hedge contract using the instrument id from the transaction
        hedge_contract = json.loads(api_factory.build(lusid.api.InstrumentsApi).get_instrument(
            identifier_type='LusidInstrumentId',
            identifier=hedge_transaction.instrument_uid
        ).instrument_definition.content)

        # Get the spot rate to buy the portfolio currency
        spot_rate = 1 / fx_rate

        # Get the interest rate for USD
        interest_rate_1 = interest_rates.loc[
            interest_rates['currency'] == 'USD', 'interest_rate'].values[0]

        # Get the interest rate for the portfolio currency
        interest_rate_2 = interest_rates.loc[
            interest_rates['currency'] == currency, 'interest_rate'].values[0]

        # Get the time in days left on the hedge contract
        time = ((parse(timestr=hedge_contract['settlement_date']) \
                - valuation_date).days) / 365

        # Get the strike price of the hedge contract
        strike_price = hedge_contract['strike_price']

        # Work out the forward FX price
        forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1) * time)

        # Work out the value of the hedge per unit of currency
        value = (forward_price - strike_price) * math.exp(-interest_rate_2 * time)

        # Work out the total value of the hedge
        total_value = value * -hedge_contract['leg1']['amount']

        # Print the value
        prettyprint.heading(
            'USD/' + currency+' forward contract value', '{} {}'.format(
                round(total_value,2), currency))

        # Upsert the quote against the hedging transaction
        quote_requests[hedge_transaction.instrument_uid] = models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider='DataScope',
                    instrument_id=hedge_transaction.instrument_uid,
                    instrument_id_type='LusidInstrumentId',
                    quote_type='Price',
                    field='Mid'),
                effective_at=transaction_date
            ),
            metric_value=models.MetricValue(
                value=total_value,
                unit=currency),
            lineage='InternalSystem')

    response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
        scope=scope,
        request_body=quote_requests)

    return prettyprint.upsert_quotes_response(response)

With a vauluation functioned defined you can now value the forward contracts.

*Run the cell below to value the forward contracts*

In [28]:
value_currency_forwards(
    scope=scope, 
    fund_currencies=fund_currencies, 
    transaction_date=holdings_effective_date,
    valuation_date=holdings_effective_date,
    interest_rates=interest_rates)

USD/AUD forward contract value: 1928.31 AUD
USD/GBP forward contract value: 2243.65 GBP
USD/EUR forward contract value: 3931.48 EUR
USD/JPY forward contract value: 450009.29 JPY


,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:05.224632+00:00,None,DataScope,None,LUID_XSQRTPVG,LusidInstrumentId,Price,Mid,"1,928.31",AUD,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:05.224632+00:00,None,DataScope,None,LUID_HNJUECZO,LusidInstrumentId,Price,Mid,"2,243.65",GBP,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:05.224632+00:00,None,DataScope,None,LUID_3SAI7Q01,LusidInstrumentId,Price,Mid,"3,931.48",EUR,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:05.224632+00:00,None,DataScope,None,LUID_T0PNV9AK,LusidInstrumentId,Price,Mid,"450,009.29",JPY,Success


## b) Valuing the foreign currency funds

With the forward contracts valued and quotes upserted against each contract for the current value you can now value the foreignh currency portfolios.

*Run the cell below to value the foreign currency portfolios*

In [29]:
currency_portfolio_values = []

inline_recipe_complex = inline_recipe
inline_recipe_complex.pricing = models.PricingContext(
    options=models.PricingOptions(
        allow_any_instruments_with_sec_uid_to_price_off_lookup=True))

for currency, currency_portfolio_code in fund_currencies.items():
    
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe_complex,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Portfolio/default/Name',
            op='Value')
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

    response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
        scope=scope,
        code=currency_portfolio_code,
        aggregation_request=aggregation_request)

    currency_portfolio_values.append(response)

currency_portfolio_valuation_one = prettyprint.aggregation_responses_generic_df(currency_portfolio_values)
currency_portfolio_valuation_one

,Sum(Holding/default/PV),Portfolio/default/Name,currency
0,"11,808,817.20",AUD offering of UK_High_Growth_Equities_Fund_b...,AUD
1,"6,491,526.09",GBP offering of UK_High_Growth_Equities_Fund_b...,GBP
2,"8,500,960.00",USD offering of UK_High_Growth_Equities_Fund_b...,USD
3,"7,460,913.93",EUR offering of UK_High_Growth_Equities_Fund_b...,EUR
4,"945,001,120.40",JPY offering of UK_High_Growth_Equities_Fund_b...,JPY


## 11) Adding additional transactions to the base fund

Perhaps you'd like to add some transactions to the base fund. In this case you are going to add USD$1,000,000 which has come as the result of dividends and coupon payments across the base fund.

*Run the cell below to add the transaction*

In [30]:
transaction_date = holdings_effective_date+timedelta(days=1)

base_fund_transactions = []

base_fund_transactions.append(
    models.TransactionRequest(
        transaction_id=1234567,
        type='FundsIn',
        instrument_identifiers={
            'Instrument/default/Currency': 'USD'},
        transaction_date=transaction_date,
        settlement_date=transaction_date,
        units=1000000,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=1000000,
            currency=currency),
        transaction_currency='USD',
        source='Client',
    )
)

# Upsert the transaction into LUSID
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=base_portfolio_code,
    transaction_request=base_fund_transactions
    )

prettyprint.transactions_response(response, scope, base_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_38b3-6f12-50d0-02
Code: 77f82cd2-3ae6-49bb-829c-021d6193eff3
Transactions Effective From: 2020-07-26 02:13:53.451114+00:00
Transactions Created On: 2020-07-29 02:14:09.030102+00:00



## 12) Re-valuing the base fund

You can now re-value the base fund with these additional funds added. As a day has passed since you last performed an aggregation we need to make sure you have up to date market data.

*Run the cell below to import the latest market quote data*

In [31]:
# Import market data for the day that the holdings of the fund are effective from
prices = pd.read_csv('data/multiplecurrencies-prices2.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,"1,625.78",close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,165.93,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.65,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,143.68,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


*Run the cell below to upsert quotes from the market data*

In [32]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=prices, 
    date=transaction_date, 
    instrument_identifier='instrument_figi', 
    lusid_identifier='Figi', 
    quote_type='price')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.104995+00:00,None,DataScope,None,BBG000BVPXP1,Figi,Price,Mid,"1,625.78",USD,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.104995+00:00,None,DataScope,None,BBG000B9XVV8,Figi,Price,Mid,165.93,USD,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.104995+00:00,None,DataScope,None,BBG00FN3B5K8,Figi,Price,Mid,98.65,USD,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.104995+00:00,None,DataScope,None,BBG000DQQNJ8,Figi,Price,Mid,143.68,USD,Success


*Run the cell below to re-value your base fund*

In [33]:
# Create the aggregation request, using the recipe defined earlier
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=transaction_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='Value'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

# Call LUSID to aggregate and value the base fund
response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
    scope=scope,
    code=base_portfolio_code,
    aggregation_request=aggregation_request)

# Set the total value and total cost of the base fund when it was first created
base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

prettyprint.heading("Base Fund Total Value", str(base_fund_total_value))
prettyprint.heading("Base Fund Total Cost", str(base_fund_total_cost))

Base Fund Total Value: 43488203.18
Base Fund Total Cost: 44109507.87


You can see here that the value of the fund has increased since the last valuation. You can now calculate the new unit price of the base fund and add this to the quote store.

*Run the cell below to upsert the updated unit price for the base fund to the quote store*

In [34]:
# Get the base fund instrument
base_fund_instrument = api_factory.build(lusid.api.InstrumentsApi).get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_code,
    property_keys=[circulation_property_key])

# Get the unique Lusid Instrument Id from the base fund instrument
luid = base_fund_instrument.lusid_instrument_id
# Get the total circulation from the base fund instrument
base_fund_total_circulation = round(base_fund_instrument.properties[0].value.metric_value.value, 0)
# Calculate the unit price of the base fund from the total value and the total circulation
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        quote_series_id=models.QuoteSeriesId(
            provider='DataScope',
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            field='Mid'),
        effective_at=transaction_date,
    ),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    lineage='InternalSystem')


# Call LUSID to upsert the quote
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=scope,
    request_body={'base_fund': instrument_quote})

# Pretty print the total value, circulation and unit price of the fund
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
# Pretty print the response from LUSID

prettyprint.upsert_quotes_response(response)

Base Fund Unit Price: 1087.21


,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.731562+00:00,None,DataScope,None,LUID_X5M03SOH,LusidInstrumentId,Price,Mid,"1,087.21",USD,Success


## 13) Re-valuing the foreign currency funds to see how the transaction on the base fund is propogated

You can now re-value your foreign currency portfolios to see how the change in the base fund has affected them. Once again you need to import fresh market data to perform a valuation.

*Run the cell below to import up to date FX rates*

In [35]:
# Import the FX rates
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates2.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.32
1,AUD/USD,0.71
2,EUR/USD,1.17
3,JPY/USD,0.01


*Run the cell below to upsert the FX quotes into the quote store*

In [36]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=fx_rates, 
    date=transaction_date, 
    instrument_identifier='pair', 
    lusid_identifier='CurrencyPair', 
    quote_type='fx_rate')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.810195+00:00,None,DataScope,None,GBP/USD,CurrencyPair,Rate,Mid,1.32,rate,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.810195+00:00,None,DataScope,None,AUD/USD,CurrencyPair,Rate,Mid,0.71,rate,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.810195+00:00,None,DataScope,None,EUR/USD,CurrencyPair,Rate,Mid,1.17,rate,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.810195+00:00,None,DataScope,None,JPY/USD,CurrencyPair,Rate,Mid,0.01,rate,Success


*Run the cell below to also create the inverse FX rate pairs*

In [37]:
fx_rates_inverse = fx_rates.copy(deep=True)
fx_rates_inverse['pair'] = fx_rates_inverse['pair'].apply(lambda x: '/'.join(x.split('/')[::-1]))
fx_rates_inverse['rate'] = fx_rates_inverse['rate'].apply(lambda x: 1/x)
fx_rates_inverse.head()

,pair,rate
0,USD/GBP,0.76
1,USD/AUD,1.41
2,USD/EUR,0.85
3,USD/JPY,112.36


*Run the cell below to upsert the inverse FX rate pairs*

In [38]:
upsert_quotes_from_data_frame(
    scope=scope,
    data_frame=fx_rates_inverse, 
    date=transaction_date, 
    instrument_identifier='pair', 
    lusid_identifier='CurrencyPair', 
    quote_type='fx_rate')

,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.890164+00:00,None,DataScope,None,USD/GBP,CurrencyPair,Rate,Mid,0.76,rate,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.890164+00:00,None,DataScope,None,USD/AUD,CurrencyPair,Rate,Mid,1.41,rate,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.890164+00:00,None,DataScope,None,USD/EUR,CurrencyPair,Rate,Mid,0.85,rate,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:09.890164+00:00,None,DataScope,None,USD/JPY,CurrencyPair,Rate,Mid,112.36,rate,Success


*Run the cell below to import the latest interest rate data to use in valuing the currency forwards*

In [39]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates2.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.03
1,AUD,0.02
2,EUR,-0.00
3,JPY,0.00
4,GBP,0.01


*Run the cell below to value the currency forward contracts and upsert their values to the quote store*

In [40]:
value_currency_forwards(
    scope=scope, 
    fund_currencies=fund_currencies, 
    transaction_date=holdings_effective_date,
    valuation_date=transaction_date,
    interest_rates=interest_rates)

USD/AUD forward contract value: -167015.7 AUD
USD/GBP forward contract value: 51742.68 GBP
USD/EUR forward contract value: 197191.88 EUR
USD/JPY forward contract value: -10378433.65 JPY


,_lineage,_cut_label,_uploaded_by,_as_at,discriminator,_provider,_price_source,_instrument_id,_instrument_id_type,_quote_type,_field,_value,_unit,status
0,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:11.320708+00:00,None,DataScope,None,LUID_XSQRTPVG,LusidInstrumentId,Price,Mid,"-167,015.70",AUD,Success
1,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:11.320708+00:00,None,DataScope,None,LUID_HNJUECZO,LusidInstrumentId,Price,Mid,"51,742.68",GBP,Success
2,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:11.320708+00:00,None,DataScope,None,LUID_3SAI7Q01,LusidInstrumentId,Price,Mid,"197,191.88",EUR,Success
3,InternalSystem,,00u4edwdnnCS5aFsN2p7,2020-07-29 02:14:11.320708+00:00,None,DataScope,None,LUID_T0PNV9AK,LusidInstrumentId,Price,Mid,"-10,378,433.65",JPY,Success


With up to date quotes populated you can now value your foreign currency funds. Here you can see how the change in the base fund has been propogated to each of the foreign currency portfolios.

*Run the cell below to get the updated value of each of the foreign currency funds*

In [41]:
currency_portfolio_values = []

for currency, currency_portfolio_code in fund_currencies.items():
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe_complex,
        effective_at=transaction_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Portfolio/default/Name',
            op='Value')
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

    response = api_factory.build(lusid.api.AggregationApi).get_aggregation(
        scope=scope,
        code=currency_portfolio_code,
        aggregation_request=aggregation_request)

    currency_portfolio_values.append(response)
    
currency_portfolio_valuation_two = prettyprint.aggregation_responses_generic_df(currency_portfolio_values)
currency_portfolio_valuation_two['Sum(Holding/default/PV) (Original)'] = currency_portfolio_valuation_one['Sum(Holding/default/PV)']
currency_portfolio_valuation_two['Increase in PV'] = currency_portfolio_valuation_two['Sum(Holding/default/PV)'] - currency_portfolio_valuation_two['Sum(Holding/default/PV) (Original)']
currency_portfolio_valuation_two[[
    'Portfolio/default/Name',
    'Sum(Holding/default/PV) (Original)',
    'Sum(Holding/default/PV)',
    'Increase in PV']]

,Portfolio/default/Name,Sum(Holding/default/PV) (Original),Sum(Holding/default/PV),Increase in PV
0,AUD offering of UK_High_Growth_Equities_Fund_b...,"11,808,817.20","12,083,237.82","274,420.62"
1,GBP offering of UK_High_Growth_Equities_Fund_b...,"6,491,526.09","6,640,894.20","149,368.11"
2,USD offering of UK_High_Growth_Equities_Fund_b...,"8,500,960.00","8,697,680.00","196,720.00"
3,EUR offering of UK_High_Growth_Equities_Fund_b...,"7,460,913.93","7,631,106.41","170,192.48"
4,JPY offering of UK_High_Growth_Equities_Fund_b...,"945,001,120.40","966,888,982.08","21,887,861.68"
